In [4]:
import numpy as np
import tensorflow.contrib.distributions as ds
from sklearn.utils import shuffle
import tensorflow as tf
import argparse
import ram

/anaconda2/envs/deeprl_py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
sess = tf.InteractiveSession()

In [6]:
# side of full glimpse will actually be glimpse_size * 2 + 1
glimpse_size = 4
num_resolutions = 3
batch_size = 9
length = 100
num_tfs = 2

In [84]:
def make_dna_seq(batch_size, length):
    one_hot_bases = np.eye(4)
    sample_indices = np.random.randint(0, 4, [batch_size, length])
    return one_hot_bases[sample_indices]

def make_atac_seq(batch_size, length):
    return np.random.randint(0, 20, [batch_size, length, 1])
    
def make_chip_seq(batch_size, num_tfs):
    return np.random.randint(0, 2, [batch_size, num_tfs])

def get_glimpses(data, num_resolutions):
    # first four channels of data are the one-hot encoded dna sequence
    dna = data[:, :, :4]
    data = data[:, :, 4:]
    glimpses = []
    print(data.shape)
    for i in range(num_resolutions):
        resolution = 2**i
        glimpse = tf.nn.pool(
            input=data,
            window_shape=[resolution],
            strides=[resolution],
            pooling_type='MAX',
            padding='SAME')
        glimpses.append(glimpse)
    res = index_glimpses(dna, location, num_resolutions, glimpses, glimpse_size, length, batch_size)
    return index_glimpses(dna, location, num_resolutions, glimpses, glimpse_size, length, batch_size)

def get_batch_glimpses(padded_glimpse, start_index, glimpse_size):
    adder = [i for i in range(2*glimpse_size)]
    indices = tf.map_fn(lambda index: index + adder, start_index)
    indices = tf.expand_dims(indices, -1)
    ind = np.repeat(np.arange(batch_size, dtype=int), glimpse_size*2).reshape([batch_size, glimpse_size*2])
    indices_index = tf.cast(tf.constant(ind), tf.int32)
    indices_index = tf.expand_dims(indices_index, -1)
    indices = tf.concat([indices_index, indices], axis = -1)
    batched_glimpses = tf.gather_nd(tf.squeeze(padded_glimpse), indices)
    batched_glimpses = tf.expand_dims(batched_glimpses, axis = -1)
    return batched_glimpses
    
def index_glimpses(dna, location, num_resolutions, glimpses, glimpse_size, length, batch_size):
    to_concatenate = []
    to_concatenate_orig = []
    for i in range(num_resolutions):
        glimpse = glimpses[i]
        # add glimpse_size to location_index
        # because glimpses will be padded with glimpse_size values on each side
        location_index = tf.to_int32(location / 2.0**i) + glimpse_size
        start_index = location_index - glimpse_size
        padded_glimpse = get_padded_glimspe(glimpse, glimpse_size)
        boolean_mask = get_boolean_mask(glimpse_size, start_index, glimpse.shape[1])
        batched_glimpses = get_batch_glimpses(padded_glimpse, start_index, glimpse_size)
#         print("glimpse")
#         print(batched_glimpses.eval())
        if i == 0:
            padded_dna = get_padded_dna(dna, glimpse_size)
            batched_dna_glimpses = get_batch_glimpses(padded_dna, start_index, glimpse_size)
            dna_boolean_mask = tf.squeeze(tf.stack([boolean_mask]*4, axis=2))
            sliced_dna = tf.boolean_mask(padded_dna, dna_boolean_mask)
            sliced_dna = tf.reshape(sliced_dna, [batch_size, glimpse_size * 2, 4])
            splits = tf.split(batched_dna_glimpses, num_or_size_splits=4, axis=2)
            #print("sliced")
            #print(tf.squeeze(sliced_dna).eval())
            #print(batched_dna_glimpses.eval() == tf.squeeze(sliced_dna).eval())
            for split in splits:
                to_concatenate.append(tf.squeeze(split, axis = 2))
            to_concatenate_orig.append(sliced_dna)

        sliced_glimpse = tf.boolean_mask(padded_glimpse, boolean_mask)
        sliced_glimpse = tf.reshape(sliced_glimpse, [batch_size, glimpse_size * 2, 1])
        to_concatenate_orig.append(sliced_glimpse)
        to_concatenate.append(batched_glimpses)
        #print(batched_glimpses.eval() == tf.squeeze(sliced_glimpse).eval())
    new = tf.concat(to_concatenate, axis=-1)
    old = tf.concat(to_concatenate_orig, axis=-1)
    diff = new - old
    print(diff.eval())
    return new
        
def get_boolean_mask(glimpse_size, start_index, length):
    curr_index = start_index
    padded_size = length + 2 * glimpse_size
    index_mask = tf.one_hot(indices=curr_index, depth=padded_size, axis=1)
    for i in range(glimpse_size * 2 - 1):
        curr_index += 1
        index_mask += tf.one_hot(indices=curr_index, depth=padded_size, axis=1)
    return index_mask > 0
    
def get_padded_glimspe(glimpse, glimpse_size):
    return tf.pad(glimpse, paddings=[[0, 0], [glimpse_size, glimpse_size], [0, 0]], constant_values=-1)

def get_padded_dna(dna, glimpse_size):
    return get_padded_glimspe(dna, glimpse_size)


In [85]:
dna = make_dna_seq(batch_size, length)
atac = make_atac_seq(batch_size, length)
chip = make_chip_seq(batch_size, num_tfs)
input_ = np.concatenate([dna, atac], axis=-1)
input_.shape

(9, 100, 5)

In [86]:
location = np.array([[20], [40], [99]] * 3)
get_glimpses(input_, num_resolutions).eval()


(9, 100, 1)
[[[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.

array([[[  0.,   1.,   0.,   0.,  10.,  18.,  13.],
        [  1.,   0.,   0.,   0.,   6.,  16.,  15.],
        [  0.,   0.,   1.,   0.,  15.,  10.,  18.],
        [  0.,   0.,   0.,   1.,  11.,  15.,  15.],
        [  1.,   0.,   0.,   0.,   4.,   4.,  14.],
        [  0.,   0.,   0.,   1.,   2.,  14.,  17.],
        [  0.,   0.,   0.,   1.,  14.,   6.,  12.],
        [  0.,   0.,   0.,   1.,   6.,  17.,  19.]],

       [[  0.,   1.,   0.,   0.,  15.,  17.,  15.],
        [  0.,   0.,   0.,   1.,   0.,  15.,  15.],
        [  0.,   1.,   0.,   0.,   7.,  15.,  17.],
        [  0.,   0.,   1.,   0.,  17.,  17.,  17.],
        [  0.,   0.,   0.,   1.,  13.,  13.,  18.],
        [  1.,   0.,   0.,   0.,   1.,  18.,  19.],
        [  0.,   1.,   0.,   0.,  18.,  19.,  19.],
        [  1.,   0.,   0.,   0.,  10.,  12.,  11.]],

       [[  0.,   0.,   1.,   0.,  11.,  16.,  13.],
        [  0.,   1.,   0.,   0.,   8.,  16.,  19.],
        [  0.,   1.,   0.,   0.,   8.,  11.,  16.],
        

In [74]:
ind = np.repeat(np.arange(batch_size), glimpse_size*2).reshape([batch_size, glimpse_size*2])

[[0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4]
 [5 5 5 5 5 5 5 5]
 [6 6 6 6 6 6 6 6]
 [7 7 7 7 7 7 7 7]
 [8 8 8 8 8 8 8 8]]
[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3, 3], [4, 4, 4, 4, 4, 4, 4, 4], [5, 5, 5, 5, 5, 5, 5, 5], [6, 6, 6, 6, 6, 6, 6, 6], [7, 7, 7, 7, 7, 7, 7, 7], [8, 8, 8, 8, 8, 8, 8, 8]]
